# Introduction

This notebook illustrates the basics of how to use `plottr` -- in particular, the `inspectr` and `autoplot` tools -- to live plot data in a qcodes database.

## Basic notebook setup

In [1]:
%gui qt

DBPATH = './qcodes_liveplot_demo.db'

import qcodes as qc

qc.config.core.db_location = DBPATH
qc.initialise_database()

# Launching inspectr

We can launch the inspectr tool either interactively, or from the command line in a separate process. For this example notebook it doesn't really matter which one. You can either run the launching cell below, or, from within the plottr root directory, run 

``
$ python apps/inspectr.py --dbpath=<path to .db file>
``

Note: initialization from within this notebook will show you the data only after the measurement has finished (because data generation and plotting run in the same thread). So for *real* live plotting, you should run the inspectr separately.

In [2]:
# only use if you're not launching the inspectr tool from a separate command line

from plottr.apps.inspectr import inspectr
from plottr import log as plottrlog

# make a log window as well. the plotting tools can use it for user information.
logwin = plottrlog.setupLogging(makeDialog=True)
logwin.show()

# this launches the db inspection tool
win = inspectr(DBPATH)
win.show()

We should now have two windows open; no data is yet shown if we started with a fresh .db file. 
Now, before populating the database, let's enable automatic monitoring of the dataset. To do that, enter a refresh interval (given in seconds) in the inspectr window toolbar, and enable the auto-plot option.

# Dummy experiments

## Qcodes imports (and other relevant stuff)

In [9]:
import time
import numpy as np
from qcodes import load_or_create_experiment, Measurement, Parameter

## A very simple 1D sweep

In [10]:
xvals = np.linspace(0, 10, 101)
yvals = np.sin(xvals)

def simple_1d_sweep():
    for x, y in zip(xvals, yvals):
        yield x, y
        
x = Parameter('x')
y = Parameter('y')

station = qc.Station(x, y)

In [12]:
exp = load_or_create_experiment('simple_1d_sweep', sample_name='no sample')

meas = Measurement(exp, station)
meas.register_parameter(x)
meas.register_parameter(y, setpoints=(x,))
meas.write_period = 2

with meas.run() as datasaver:
    for xval, yval in simple_1d_sweep():
        datasaver.add_result(
            (x, xval),
            (y, yval),
        )
        time.sleep(0.5)

Starting experimental run with id: 2
